# K‑MIMIC SQL Usage
**Last updated:** 2025-10-16

This notebook demonstrates how to utilize **K-MIMIC** data using SQL query


## What you will do
1. **Set up**
2. **Load and preprocess**
3. **Demographics**
4. **ICU-level analysis (age and mortality)**
5. **Analyze events using d_items**

## Requirements (minimal)
- Python 3.9+
- Common data libraries: `pandas`, `numpy`
- For SQL connection: `psycopg2`
- Visualization: `matplotlib`

# A database connection example (Postgres)

In this example, we utilize `psycopg2` and `pandas` to connect and send a query to the database server. 

In [ ]:
import psycopg2
import pandas as pd

In [ ]:
# 데이터베이스 연결 정보
DB_CONFIG = {
    'host': 'pg-2vge6u.vpc-cdb-kr.gov-ntruss.com',        # 데이터베이스 호스트
    'port': 5432,               # 포트 번호
    'database': 'kmimic', # 데이터베이스 이름 (Please make sure there is kmimic)
    'user': 'meddataforge',    # 사용자명
    'password': 'c9d3b7f21!' # 비밀번호
}

In [ ]:
def get_tables_in_schema(conn, schema_name='km001'):  # It could be replaced by 'km006' or 'km012' for your interest.
    """특정 스키마의 모든 테이블 목록 조회"""
    query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = %s 
    AND table_type = 'BASE TABLE'
    ORDER BY table_name;
    """
    
    df = pd.read_sql_query(query, conn, params=(schema_name,))
    return df['table_name'].tolist()

In [ ]:
def get_all_schemas(conn):
    """데이터베이스의 모든 스키마 목록 조회"""
    query = """
    SELECT schema_name 
    FROM information_schema.schemata
    WHERE schema_name NOT IN ('information_schema', 'pg_catalog', 'pg_toast')
    ORDER BY schema_name;
    """
    
    df = pd.read_sql_query(query, conn)
    return df['schema_name'].tolist()

In [ ]:
def main():
    try:
        # 데이터베이스 연결
        print("데이터베이스에 연결 중...")
        conn = psycopg2.connect(**DB_CONFIG)
        print("연결 성공!\n")
        
        # 모든 스키마 목록 먼저 확인
        print("데이터베이스의 모든 스키마 목록:")
        schemas = get_all_schemas(conn)
        for i, schema in enumerate(schemas, 1):
            print(f"{i}. {schema}")
        
        print("\n" + "=" * 80 + "\n")
        
        # km001 스키마의 테이블 목록 조회
        schema_name = 'km001'
        print(f"'{schema_name}' 스키마의 테이블 목록:")
        tables = get_tables_in_schema(conn, schema_name)
        
        if not tables:
            print(f"'{schema_name}' 스키마에 테이블이 없습니다.")
            return
        
        for i, table in enumerate(tables, 1):
            print(f"{i}. {table}")
       
        
    except psycopg2.Error as e:
        print(f"데이터베이스 오류: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        if 'conn' in locals() and conn:
            conn.close()
            print("\n데이터베이스 연결 종료")

In [ ]:
main()  # This print all the tables in K-MIMIC. You can figure out which tables are useful to solve your problem exploring the tables.

# Basic example: Showing and counting patients

In [ ]:
def get_patients(conn, schema_name='km001'):
    """특정 스키마의 모든 테이블 목록 조회"""
    query = f"""
    SELECT * FROM {schema_name}.patients LIMIT 10;
    """
    
    df = pd.read_sql_query(query, conn, params=(schema_name,))
    return df

def count_patients(conn, schema_name='km001'):
    query = f"""
    SELECT COUNT(*) as count FROM {schema_name}.patients ORDER BY count;
    """
    df = pd.read_sql_query(query, conn, params=(schema_name,))
    return df

conn = psycopg2.connect(**DB_CONFIG)

In [ ]:
get_patients(conn, 'km001')  # Change km001 to the other hospital ids (e.g., km006, km012)

In [ ]:
count_patients(conn, 'km001')

# An example of ICU-based analysis

## Listing ICU names in the hospital

In [ ]:
def list_icu_names(conn, schema_name='km001'):
    query = f"""
    SELECT
        first_careunit,
        COUNT(*) as total_icustays,
        COUNT(DISTINCT subject_id) as unique_patients,
        COUNT(DISTINCT hadm_id) as unique_admissions
    FROM {schema_name}.icustays
    GROUP BY first_careunit
    ORDER BY total_icustays DESC;
    """

    df = pd.read_sql_query(query, conn, params=(schema_name,))
    return df

conn = psycopg2.connect(**DB_CONFIG)

In [ ]:
list_icu_names(conn, 'km001')

## Analyze and plot age information

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def parse_age_and_plot_distribution(conn, schema_name='km001'):
    """ICU별 환자 나이 분포 분석 및 시각화 (matplotlib only)"""
    
    # 1단계: 데이터 가져오기
    query = f"""
    SELECT
        i.first_careunit,
        i.subject_id,
        p.anchor_age,
        p.sex
    FROM {schema_name}.icustays i
    INNER JOIN {schema_name}.patients p
        ON i.subject_id = p.subject_id
    """
    
    df = pd.read_sql_query(query, conn)
    
    print("Original age format sample:")
    print(df['anchor_age'].head(10))
    print(f"\nTotal records: {len(df)}")
    
    # 2단계: 나이를 숫자로 변환
    def parse_age(age_str):
        """'64 years, 10 months, 5 days' 형식을 숫자(년)로 변환"""
        if pd.isna(age_str) or age_str == '':
            return None
        
        try:
            age_years = 0
            age_str = str(age_str).lower()
            
            # years 추출
            if 'year' in age_str:
                years = age_str.split('year')[0].strip().split()[-1]
                age_years += float(years)
            
            # months 추출 (년으로 변환)
            if 'month' in age_str:
                months_part = age_str.split('month')[0].strip()
                if ',' in months_part:
                    months = months_part.split(',')[-1].strip()
                else:
                    months = months_part.split()[-1]
                age_years += float(months) / 12
            
            # days 추출 (년으로 변환)
            if 'day' in age_str:
                days_part = age_str.split('day')[0].strip()
                if ',' in days_part:
                    days = days_part.split(',')[-1].strip()
                else:
                    days = days_part.split()[-1]
                age_years += float(days) / 365
            
            return round(age_years, 1)
        except:
            return None
    
    df['age_numeric'] = df['anchor_age'].apply(parse_age)
    
    # 결측치 제거
    df_clean = df.dropna(subset=['age_numeric'])
    
    print(f"\nAfter parsing - valid records: {len(df_clean)}")
    print(f"Age range: {df_clean['age_numeric'].min():.1f} - {df_clean['age_numeric'].max():.1f} years")
    print(f"\nAge statistics by ICU:")
    print(df_clean.groupby('first_careunit')['age_numeric'].describe())
    
    # 3단계: 시각화
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Box plot - ICU별 나이 분포
    ax1 = axes[0, 0]
    icu_list = sorted(df_clean['first_careunit'].unique())
    age_data = [df_clean[df_clean['first_careunit'] == icu]['age_numeric'].values 
                for icu in icu_list]
    
    bp1 = ax1.boxplot(age_data, labels=icu_list, patch_artist=True)
    for patch in bp1['boxes']:
        patch.set_facecolor('lightblue')
    ax1.set_title('Age Distribution by ICU (Box Plot)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('ICU Care Unit', fontsize=12)
    ax1.set_ylabel('Age (years)', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # 2. Histogram overlaid - ICU별 나이 히스토그램
    ax2 = axes[0, 1]
    colors = plt.cm.tab10(range(len(icu_list)))
    for i, icu in enumerate(icu_list):
        icu_data = df_clean[df_clean['first_careunit'] == icu]['age_numeric']
        ax2.hist(icu_data, alpha=0.5, label=icu, bins=20, color=colors[i])
    ax2.set_title('Age Histogram by ICU', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Age (years)', fontsize=12)
    ax2.set_ylabel('Frequency', fontsize=12)
    ax2.legend(loc='upper right')
    ax2.grid(alpha=0.3)
    
    # 3. 평균 나이 Bar chart
    ax3 = axes[1, 0]
    age_stats = df_clean.groupby('first_careunit')['age_numeric'].agg(['mean', 'std', 'count'])
    age_stats = age_stats.sort_values('mean', ascending=False)
    
    x_pos = range(len(age_stats))
    colors_bar = plt.cm.viridis(age_stats['mean']/age_stats['mean'].max())
    bars = ax3.bar(x_pos, age_stats['mean'], yerr=age_stats['std'], 
                   capsize=5, color=colors_bar, alpha=0.8)
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(age_stats.index, rotation=45, ha='right')
    ax3.set_title('Mean Age by ICU (with Standard Deviation)', fontsize=14, fontweight='bold')
    ax3.set_xlabel('ICU Care Unit', fontsize=12)
    ax3.set_ylabel('Mean Age (years)', fontsize=12)
    ax3.grid(axis='y', alpha=0.3)
    
    # 각 막대 위에 값 표시
    for i, (bar, mean, count) in enumerate(zip(bars, age_stats['mean'], age_stats['count'])):
        ax3.text(bar.get_x() + bar.get_width()/2, mean + age_stats['std'].iloc[i] + 1,
                f'{mean:.1f}y\n(n={count})', 
                ha='center', va='bottom', fontsize=9)
    
    return df_clean, age_stats

# 실행
df_age, age_stats = parse_age_and_plot_distribution(conn)
print("\n=== Summary Statistics ===")
print(age_stats)

## Mortality analysis for each ICU units

In [ ]:
def plot_simple_mortality_chart(conn, schema_name='km001'):
    """Hospital mortality rate만 표시하는 간단한 버전"""
    
    query = f"""
    SELECT
        i.first_careunit,
        COUNT(*) as total_icustays,
        ROUND(100.0 * SUM(CASE WHEN a.hospital_expire_flag = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) as mortality_rate
    FROM {schema_name}.icustays i
    LEFT JOIN {schema_name}.admissions a 
        ON i.subject_id = a.subject_id 
        AND i.hadm_id = a.hadm_id
    GROUP BY i.first_careunit
    ORDER BY mortality_rate DESC;
    """
    
    df = pd.read_sql_query(query, conn)
    
    # 그래프 생성
    plt.figure(figsize=(12, 6))
    bars = plt.bar(df['first_careunit'], df['mortality_rate'], 
                   color=plt.cm.RdYlGn_r(df['mortality_rate']/df['mortality_rate'].max()))
    
    plt.title('Hospital Mortality Rate by ICU Care Unit', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('ICU Care Unit', fontsize=12)
    plt.ylabel('Mortality Rate (%)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # 각 막대 위에 값 표시
    for i, (bar, value) in enumerate(zip(bars, df['mortality_rate'])):
        plt.text(bar.get_x() + bar.get_width()/2, value + 0.5, 
                f'{value}%\n(n={df.iloc[i]["total_icustays"]})', 
                ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return df

# 실행
mortality_df = plot_simple_mortality_chart(conn)

# Nursing record analysis in datetimeevents using D_ITEMS (metadata)

### NOTE: Check your Korean font setting. Generated plots could print broken characters becasue the records in datetimeevents are (usually) written in Korean unless Korean font is set in the right way.

In [ ]:
def analyze_datetimeevents_comprehensive(conn, schema_name='km001'):
    """datetimeevents 종합 분석 (d_items와 조인)"""
    
    # 1. 상위 이벤트 분석
    query_top_events = f"""
    SELECT 
        de.itemid,
        di.label,
        di.category,
        COUNT(*) as event_count,
        COUNT(DISTINCT de.subject_id) as unique_patients,
        COUNT(DISTINCT de.hadm_id) as unique_admissions,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
    FROM {schema_name}.datetimeevents de
    LEFT JOIN {schema_name}.d_items di
        ON de.itemid = di.itemid
    GROUP BY de.itemid, di.label, di.category
    ORDER BY event_count DESC
    LIMIT 30
    """
    
    df_top = pd.read_sql_query(query_top_events, conn)
    
    print("=== Top 30 Datetime Events ===")
    print(df_top)
    
    # 2. 기본 통계
    query_stats = f"""
    SELECT 
        COUNT(*) as total_events,
        COUNT(DISTINCT subject_id) as unique_patients,
        COUNT(DISTINCT hadm_id) as unique_admissions,
        COUNT(DISTINCT stay_id) as unique_stays,
        COUNT(DISTINCT itemid) as unique_item_types,
        MIN(charttime) as earliest_event,
        MAX(charttime) as latest_event
    FROM {schema_name}.datetimeevents
    """
    
    stats = pd.read_sql_query(query_stats, conn)
    print("\n=== Overall Statistics ===")
    print(stats)
    
    # 3. 시각화
    fig = plt.figure(figsize=(18, 12))
    
    # 3-1. 상위 20개 이벤트 빈도 (horizontal bar)
    ax1 = plt.subplot(2, 2, 1)
    top20 = df_top.head(20)
    colors = plt.cm.viridis(range(len(top20)))
    bars = ax1.barh(range(len(top20)), top20['event_count'], color=colors)
    ax1.set_yticks(range(len(top20)))
    ax1.set_yticklabels(top20['label'], fontsize=9)
    ax1.set_xlabel('Event Count', fontsize=12)
    ax1.set_title('Top 20 Most Frequent Datetime Events', fontsize=14, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    # 값 표시
    for i, (bar, count) in enumerate(zip(bars, top20['event_count'])):
        ax1.text(count + 5000, bar.get_y() + bar.get_height()/2, 
                f'{count:,}', va='center', fontsize=8)
    
    # 3-2. 환자 수 (horizontal bar)
    ax2 = plt.subplot(2, 2, 2)
    colors2 = plt.cm.plasma(range(len(top20)))
    bars2 = ax2.barh(range(len(top20)), top20['unique_patients'], color=colors2)
    ax2.set_yticks(range(len(top20)))
    ax2.set_yticklabels(top20['label'], fontsize=9)
    ax2.set_xlabel('Number of Patients', fontsize=12)
    ax2.set_title('Top 20 Events by Patient Count', fontsize=14, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    
    # 값 표시
    for i, (bar, count) in enumerate(zip(bars2, top20['unique_patients'])):
        ax2.text(count + 200, bar.get_y() + bar.get_height()/2, 
                f'{count:,}', va='center', fontsize=8)
    
    # 3-3. 비율 pie chart (상위 10개 + 기타)
    ax3 = plt.subplot(2, 2, 3)
    top10 = df_top.head(10)
    others_pct = 100 - top10['percentage'].sum()
    
    labels = [label[:30] + '...' if len(label) > 30 else label 
              for label in top10['label']]
    labels.append('Others')
    
    sizes = list(top10['percentage']) + [others_pct]
    colors_pie = plt.cm.Set3(range(len(sizes)))
    
    wedges, texts, autotexts = ax3.pie(sizes, labels=labels, autopct='%1.1f%%',
                                         startangle=90, colors=colors_pie)
    for text in texts:
        text.set_fontsize(9)
    for autotext in autotexts:
        autotext.set_fontsize(8)
        autotext.set_color('white')
        autotext.set_weight('bold')
    
    ax3.set_title('Event Distribution (Top 10 + Others)', fontsize=14, fontweight='bold')
    
    # 3-4. 환자당 평균 이벤트 수
    ax4 = plt.subplot(2, 2, 4)
    top20['events_per_patient'] = top20['event_count'] / top20['unique_patients']
    top15_avg = top20.head(15).sort_values('events_per_patient', ascending=True)
    
    colors4 = plt.cm.coolwarm(top15_avg['events_per_patient'] / top15_avg['events_per_patient'].max())
    bars4 = ax4.barh(range(len(top15_avg)), top15_avg['events_per_patient'], color=colors4)
    ax4.set_yticks(range(len(top15_avg)))
    ax4.set_yticklabels(top15_avg['label'], fontsize=9)
    ax4.set_xlabel('Average Events per Patient', fontsize=12)
    ax4.set_title('Top 15 Events by Frequency per Patient', fontsize=14, fontweight='bold')
    ax4.grid(axis='x', alpha=0.3)
    
    # 값 표시
    for i, (bar, avg) in enumerate(zip(bars4, top15_avg['events_per_patient'])):
        ax4.text(avg + 0.5, bar.get_y() + bar.get_height()/2, 
                f'{avg:.1f}', va='center', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    return df_top, stats

# 실행
df_events, stats = analyze_datetimeevents_comprehensive(conn)